# 감성 분석 모델 학습 및 추론

In [2]:
import json
import glob
import pandas as pd
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

### 1. 데이터 로드

In [3]:
# 데이터 다운로드
train_files = glob.glob('data/train/*.json')
val_files = glob.glob('data/val/*.json')
# for f in file_list:
#     with open(f, encoding='utf-8') as fp:
#         data = json.load(fp)
#         for r in data:
#             records.append((r['RawText'], int(r['GeneralPolarity'])))
# r['GeneralPolarity']를 못 찾는 에러 발생해서 수정함.
# 훈련/평가 데이터를 합쳐서 가져오는 바람에 또 수정.

def load_json(files):
    records = []
    for f in files:
        with open(f, encoding='utf-8') as fp:
            data = json.load(fp)
            for r in data:
                text = r.get('RawText')
                polarity = r.get('GeneralPolarity')
                if text is not None and polarity is not None:
                    polarity = int(polarity)
                    records.append((text, polarity))
    # 데이터프레임 생성
    return pd.DataFrame(records, columns=['text', 'label'])

train_df = load_json(train_files)
val_df = load_json(val_files)

In [4]:
print(len(train_df))
print(len(val_df))
print(train_df.head())
print(val_df.head())

15229
1974
                                                text  label
0                                  가격이 착하고 디자인이 예쁩니다      1
1                    싸고  디자인이 예뻐요. . 정말  가성비 가심비 입니다      1
2  편하고  디자인이 예뻐요  가격도  좋아요   시원해요  빨리 마르고  이것만  입게되요      1
3                          너무 착한가격에 감사합니다 윈하는 색은 없지만      1
4  가격이  너무 좋아서  블랙 구매했습니다  그런데 소재도  맘에  들어  흰색도  ...      1
                                                text  label
0  엄마 사드렸는데 배도 편하게 눌러주고  미디가 길어 편하다고...다섯가지 색상 모두...      1
1  입자마자 시원하고 배 부분도 잘 잡아주고 입어보니 넝수편하네요^^ 반사이즈는 아래가...      1
2               생각처럼 얇지는 않지만 스판기짱에 착용감이 완전편하고 좋아요 굿굿      1
3  장점은 품질.핏은 좋아요 가격은 싸요 단점은 더워요~~허리밴드부분이 땀 채여요 ㅜㅜ...      0
4    허리를 강하게 잡아주어서 좋구요.원단이 앏아서 덥지 않는 바지입니다. 맘에듭니다~^^      1


### 2. 데이터 전처리

In [5]:
# 결측치 제거
# 한글 토큰화 전처리 (특수문자 처리, 어간 추출, 불용어 처리) -> 함수

okt = Okt()
stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def kor_tokenize(text):
    tokens = okt.morphs(text, stem = True)
    tokens = [t for t in tokens if t not in stop_words]
    return ' '.join(tokens)

# 훈련데이터 전처리
# 평가데이터 전처리
train_df.dropna(inplace=True)
val_df.dropna(inplace=True)

train_df['new_text'] = train_df['text'].apply(kor_tokenize)
val_df['new_text'] = val_df['text'].apply(kor_tokenize)

In [11]:
# 라벨 변환 (-1, 0, 1 을 0, 1, 2로)
train_df['label_add'] = train_df['label'] + 1
val_df['label_add'] = val_df['label'] + 1

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['new_text'])

X_train = tokenizer.texts_to_sequences(train_df['new_text'])
X_val = tokenizer.texts_to_sequences(val_df['new_text'])

max_len = 50
X_train = pad_sequences(X_train, maxlen=max_len)
X_val = pad_sequences(X_val, maxlen = max_len)

y_train = train_df['label_add'].values
y_val = val_df['label_add'].values

### 3. 모델 정의 및 생성

In [14]:
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_shape = (max_len,)))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))  # -1, 0, 1 

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


c:\Users\Playdata\anaconda3\envs\nlp_env\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 50, 128)        │     1,224,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,364,611 (5.21 MB)

 Trainable params: 1,364,611 (5.21 MB)

 Non-trainable params: 0 (0.00 B)

### 4. 모델 학습

In [15]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_val, y_val)
)

Epoch 1/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 14s 54ms/step - accuracy: 0.7322 - loss: 0.6193 - val_accuracy: 0.7487 - val_loss: 0.5755
Epoch 2/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 12s 52ms/step - accuracy: 0.8291 - loss: 0.4116 - val_accuracy: 0.7634 - val_loss: 0.5891
Epoch 3/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.8682 - loss: 0.3338 - val_accuracy: 0.7614 - val_loss: 0.5829
Epoch 4/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 12s 52ms/step - accuracy: 0.8942 - loss: 0.2771 - val_accuracy: 0.7508 - val_loss: 0.6796
Epoch 5/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - accuracy: 0.9095 - loss: 0.2383 - val_accuracy: 0.7558 - val_loss: 0.7721
Epoch 6/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.9278 - loss: 0.1972 - val_accuracy: 0.7447 - val_loss: 0.7974
Epoch 7/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - accuracy: 0.9366 - loss: 0.1723 - val_accuracy: 0.7396 - val_loss: 0.9358
Epoch 8/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - accuracy: 0.9460 - loss: 0.1455 - 

### 5. 추론

In [18]:
def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences([kor_tokenize(text)])
    pad_seq = pad_sequences(seq, maxlen=max_len)
    pred = model.predict(pad_seq)
    # 다시 0→-1, 1→0, 2→1
    return pred.argmax() - 1

print(predict_sentiment("색깔은 예쁜데 그냥 그래요."))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
0


In [25]:
def predict_sentiments(texts):
    results = []
    for t in texts:
        seq = tokenizer.texts_to_sequences([kor_tokenize(t)])
        pad_seq = pad_sequences(seq, maxlen=max_len)
        pred = model.predict(pad_seq, verbose=0)
        label = pred.argmax() - 1   
        results.append((t, label))
    return results

# 테스트
test_texts = [
    "가격이 착하고 디자인이 예쁩니다",
    "배송이 느리고 품질이 별로였어요",
    "그냥 무난한 상품입니다",
    "색깔은 예쁜데 그냥 그래요.",
    "와... 배송이 이렇게 빠를 줄이야! 대만족이에요", 
    "사진이랑 완전 다르고 품질이 엉망이에요",                
    "그냥 평범해요, 특별한 건 없네요",               
    "포장 상태가 너무 엉망이라 화가 났어요",          
    "할인 쿠폰 덕분에 저렴하게 샀습니다",        
    "한두 번은 쓸만한데 내구성이 약하네요"
]

for text, label in predict_sentiments(test_texts):
    print(f"문장: {text}\n예측 라벨: {label}\n")


문장: 가격이 착하고 디자인이 예쁩니다
예측 라벨: 1

문장: 배송이 느리고 품질이 별로였어요
예측 라벨: -1

문장: 그냥 무난한 상품입니다
예측 라벨: 0

문장: 색깔은 예쁜데 그냥 그래요.
예측 라벨: 0

문장: 와... 배송이 이렇게 빠를 줄이야! 대만족이에요
예측 라벨: 1

문장: 사진이랑 완전 다르고 품질이 엉망이에요
예측 라벨: -1

문장: 그냥 평범해요, 특별한 건 없네요
예측 라벨: 0

문장: 포장 상태가 너무 엉망이라 화가 났어요
예측 라벨: -1

문장: 할인 쿠폰 덕분에 저렴하게 샀습니다
예측 라벨: 1

문장: 한두 번은 쓸만한데 내구성이 약하네요
예측 라벨: -1

